# 

In [88]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs

import json
import re

import os as os
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


Web Scraping El tiempo

In [89]:
f = open('config.json')
empresas = json.load(f)

if not os.path.exists('../data/raw/eltiempo.csv'):
    os.makedirs('../data/raw/eltiempo.csv')
    

In [90]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")
paginas = input("la cantidad de paginas ")

In [91]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()

In [92]:
titulares=pd.DataFrame()
print(type(titulares))

<class 'pandas.core.frame.DataFrame'>


In [93]:
# titulares=[]
get_url = driver.current_url

# hacer el WS del primer url
bs.obtener_articulos_eltiempo_dataframe(driver=driver, url=f'https://www.eltiempo.com/buscar?q={empresa}',titulares = titulares,empresa=empresa)

# for i in range(2,int(paginas) + 1):
#         bs.obtener_articulos_eltiempo_dataframe(driver=driver, url=f'https://www.eltiempo.com/buscar/{i}?q={empresa}',titulares = titulares,empresa=empresa)


AttributeError: module 'base' has no attribute 'obtener_articulos_eltiempo_dataframe'

### Extrae info de cada URL

In [ ]:
# Inicializa los cookies del navegador 
driver.delete_all_cookies()

# Busca cada articulo y las almacena en la lista de titulares
for tit in titulares:
    
    driver.get(tit['URL'])
    driver.implicitly_wait(10) #Nueva metodología de wait


    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    contenido = ''
    try :
        html = WebDriverWait(driver,10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH,'.//div[contains(@class,"modulos public-side")]')))
        parrafos = html.find_elements(By.XPATH,'.//p') 

    except:
        try:
            html = driver.find_element(By.XPATH,'.//div[contains(@class,"modulos public-side")]')
            parrafos = html.find_elements(By.XPATH,'.//p')
        except:
            contenido = 'SIN PARRAFOS'
        else:
            for i in parrafos:
                contenido += i.text

    else:
        try:
            for i in parrafos:
                contenido += i.text
        except:
            print('Error en el último for == el más dentro de "for tit in titulares:" externo')
    
    # agregar contenido al dict de titulares
    tit['Contenido'] = contenido
    driver.delete_all_cookies() # clear all cookies in scope of session
    
    #agregar lista de URLs de noticias relacionadas
    # tit['RelNewsUrls'] = bs.obtener_articulos_relacionados_eltiempo(driver)


### Para agregar la info en un archivo CSV

In [ ]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Empresa','Tema','Contenido']


# Para AGREGAR titulares a un archivo.
with open('../data/raw/eltiempo.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [ ]:
# Imprimir los últimos datos de
print(titulares[10])

# Cerrar navegador
driver.quit()

In [ ]:
df_titulares=pd.DataFrame(titulares)
print(df_titulares.at[10,'URL'])
